In [1]:
import pandas as pd
import datetime
from db import MySqlHelper
from basic import get_date_shift, get_yesterday, to_datetime, get_today, check_is_UTC0, timing, logging_channels, date_range, datetime_to_str
from jieba_based import Composer_jieba
from keyword_usertag_report import keyword_usertag_report, delete_expired_rows
import jieba.analyse
def clean_keyword_list(keyword_list, stopwords, stopwords_usertag):
    keyword_list = Composer_jieba().clean_keyword(keyword_list, stopwords)  ## remove stopwords
    keyword_list = Composer_jieba().clean_keyword(keyword_list, stopwords_usertag)  ## remove stopwords, only for usertag
    keyword_list = Composer_jieba().filter_quantifier(keyword_list)  ## remove number+quantifier, ex: 5.1萬
    keyword_list = Composer_jieba().filter_str_list(keyword_list, pattern="[0-9]{2}")  ## remove 2 digit number
    keyword_list = Composer_jieba().filter_str_list(keyword_list, pattern="[0-9.]*")  ## remove floating
    keyword_list = Composer_jieba().filter_str_list(keyword_list, pattern="[a-z]{1,4}|[A-Z]{2}")  ## remove 1-4 lowercase letter and 2 Upper
    keyword_list = [keyword for keyword in keyword_list if keyword != ''] ## remove blank
    return keyword_list
def fetch_browse_record_join(web_id, date, is_df=False):
    date_start = to_datetime(date)
    date_end = date_start - datetime.timedelta(days=-1, seconds=1)  ## pixnet, upmedia, ctnews, cmoney,
    query = \
        f"""
            SELECT 
            s.uuid,
            t.code,
            t.registation_id AS token,
            t.cert_web_id,
            s.article_id,
            l.title,
            l.content,
            l.keywords
        FROM
            subscriber_browse_record s
                INNER JOIN
            article_list l ON s.article_id = l.signature                
                AND s.web_id = '{web_id}'                
                AND s.click_time BETWEEN '{date_start}' AND '{date_end}'
                AND l.web_id = '{web_id}'
                INNER JOIN         
            token_index t ON t.uuid = s.uuid
                AND t.invalid = 0
                AND t.web_id = '{web_id}'            
        """
    print(query)
    data = MySqlHelper('dione').ExecuteSelect(query)
    if is_df:
        df = pd.DataFrame(data, columns=['web_id', 'uuid', 'token', 'cert_web_id', 'article_id', 'title', 'content', 'keywords'])
        return df
    else:
        return data
@timing
def fetch_usertag_web_id_ex_day():
    query = "SELECT web_id, usertag_keyword_expired_day FROM web_id_table where usertag_keyword_enable=1"
    print(query)
    data = MySqlHelper('dione').ExecuteSelect(query)
    web_id_all = [d[0] for d in data]
    expired_day_all = [d[1] for d in data]
    return web_id_all, expired_day_all

def main_update_subscriber_usertag(web_id, date, is_UTC0, jump2gcp, expired_day, jieba_base, stopwords, stopwords_usertag)
    expired_date = get_date_shift(date_ref=date, days=-expired_day, to_str=True,
                                  is_UTC0=is_UTC0)  ## set to today + 3 (yesterday+4), preserve 4 days
    ####從不同表讀取一些相關資料#######
    data = fetch_browse_record_join(web_id, date=date, is_df=False)
    
    j, data_save = 0, {}
    #####將資料整理 並新增usertag,cut,webid
    for i, d in enumerate(data):
        uuid, code, token, cert_web_id, article_id, title, content, keywords = d
        news = title + ' ' + content
        ## pattern for removing https
        news_clean = jieba_base.filter_str(news, pattern="https:\/\/([0-9a-zA-Z.\/]*)")
        ## pattern for removing symbol, -,+~.
        news_clean = jieba_base.filter_symbol(news_clean)
        ##整理keywors,沒有新增
        if (keywords == '') | (keywords == '_'):
            keyword_list = jieba.analyse.extract_tags(news_clean, topK=80) # TF-IDF
            keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)[:8] #取前8關鍵字
            keywords = ','.join(keyword_list)  ## add keywords  
            is_cut = 1
        else:
            keyword_list = [k.strip() for k in keywords.split(',')]
            keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)
            is_cut = 0
        ##新增usertag
        for keyword in keyword_list:
            data_save[j] = {'web_id': web_id, 'uuid': uuid, 'code': code, 'token': token, 'cert_web_id': cert_web_id,
                            'news': news_clean, 'keywords': keywords, 'usertag': keyword, 'article_id': article_id,
                            'expired_date': expired_date, 'is_cut': is_cut}
            j += 1
        if i % 1000 == 0:
            
            
        mysqlhelper
    






if __name__ == '__main__':
    is_UTC0 = check_is_UTC0()      
    jump2gcp = True
    date = get_yesterday(is_UTC0=is_UTC0) ## compute all browsing record yesterday ad 3:10 o'clock
    date_list = [date]
    jieba_base =Composer_jieba()
    all_hashtag = jieba_base.set_config()
    stopwords = jieba_base.get_stopword_list()
    stopwords_usertag = jieba_base.read_file('./jieba_based/stop_words_usertag.txt')
    web_id_all, expired_day_all = fetch_usertag_web_id_ex_day()
    for date in date_list:
        for web_id ,expired_day in zip(web_id_all,expired_day_all):
            df_map_save, df_freq_token = main_update_subscriber_usertag(web_id, date, is_UTC0, jump2gcp, expired_day, jieba_base, stopwords, stopwords_usertag)




Building prefix dict from /home/hodo/Desktop/keyword_extraction/jieba_based/idf_POS_collect.txt ...
Loading model from cache /tmp/jieba.u8fbda33d7c83311d819c0f096452d7fd.cache
Loading model cost 0.998 seconds.
Prefix dict has been built successfully.


_load_cut_config time consuming:  3.635 seconds
_load_kw_config time consuming:  0.660 seconds
add_words time consuming:  0.004 seconds
SELECT keywords FROM article_list 
        WHERE keywords!='_' AND keywords!='' AND web_id in ('ctnews','mirrormedia','upmedia','btnet','bnext','dailyview','moneyweekly','nongnong','newscts','newtalk','setn','nownews')
fetch_all_hashtags time consuming:  24.238 seconds
add_words time consuming:  4.920 seconds
add_words time consuming:  0.264 seconds
get_stopword_list time consuming:  0.001 seconds
SELECT web_id, usertag_keyword_expired_day FROM web_id_table where usertag_keyword_enable=1
fetch_usertag_web_id_ex_day time consuming:  0.658 seconds


In [8]:
    expired_date = get_date_shift(date_ref=date, days=-4, to_str=True,
                                  is_UTC0=is_UTC0) 

In [9]:
expired_date

'2022-04-14'

In [5]:
date

datetime.datetime(2022, 4, 10, 0, 0)

In [10]:
date_start = to_datetime(date)

In [11]:
date_start

datetime.datetime(2022, 4, 10, 0, 0)

In [12]:
date

datetime.datetime(2022, 4, 10, 0, 0)

In [ ]:
news_clean = jieba_base.filter_str(news, pattern="https:\/\/([0-9a-zA-Z.\/]*)")

In [13]:
    def filter_str(self, text, pattern = "[https]{4,5}:\/\/[0-9a-zA-Z.\/]*"):
        text_clean = ''.join(re.split(pattern, text))
        return text_clean

In [15]:
import re

In [16]:

text_clean = ''.join(re.split("[https]{4,5}:\/\/[0-9a-zA-Z.\/]*", 'htttp://2131313'))

In [17]:
text_clean

''

In [18]:
my_string = "aavvccccddddeee"

# converting the string to a set
temp_set = set(my_string)

In [20]:
len(temp_set)

5